In [1]:
!pip install -q --no-index torch-scatter -f https://data.pyg.org/whl/torch-1.8.1+cu111.html
!pip install -q --no-index torch-sparse -f https://data.pyg.org/whl/torch-1.8.1+cu111.html
!pip install -q --no-index torch-cluster -f https://data.pyg.org/whl/torch-1.8.1+cu111.html
!pip install -q --no-index torch-spline-conv -f https://data.pyg.org/whl/torch-1.8.1+cu111.html
!pip install -q torch-geometric

In [4]:
!pip install -q autogl==0.3.1

In [1]:
import yaml
import random
import torch.backends.cudnn
import numpy as np
from autogl.datasets import build_dataset_from_name
from autogl.solver import AutoNodeClassifier
from autogl.module import Acc
from autogl.backend import DependentBackend

[2022-05-31 13:01:23] INFO (root/MainThread) PyTorch Lightning is not installed.
[2022-05-31 13:01:23] INFO (root/MainThread) Tensorflow is not installed.


In [58]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)
seed = 123
# set random seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
choices=[
            "cora",
            "pubmed",
            "citeseer",
            "coauthor_cs",
            "coauthor_physics",
            "amazon_computers",
            "amazon_photo",
        ] 
dataset_name = choices[0]
dataset = build_dataset_from_name(dataset_name)
label = dataset[0].nodes.data["y" if DependentBackend.is_pyg() else "label"]
num_classes = len(np.unique(label.numpy()))

In [27]:
dataset

In [4]:
spec = """ensemble:
  name: null
feature:
- name: NormalizeFeatures
hpo:
  max_evals: 10
  name: random
models:
- hp_space:
  - feasiblePoints: '2'
    parameterName: num_layers
    type: DISCRETE
  - cutFunc: lambda x:x[0] - 1
    cutPara:
    - num_layers
    length: 1
    maxValue:
    - 64
    minValue:
    - 16
    numericalType: INTEGER
    parameterName: hidden
    scalingType: LOG
    type: NUMERICAL_LIST
  - maxValue: 0.8
    minValue: 0.2
    parameterName: dropout
    scalingType: LINEAR
    type: DOUBLE
  - feasiblePoints:
    - leaky_relu
    - relu
    - elu
    - tanh
    parameterName: act
    type: CATEGORICAL
  name: gcn-model
trainer:
  hp_space:
  - maxValue: 300
    minValue: 100
    parameterName: max_epoch
    scalingType: LINEAR
    type: INTEGER
  - maxValue: 30
    minValue: 10
    parameterName: early_stopping_round
    scalingType: LINEAR
    type: INTEGER
  - maxValue: 0.05
    minValue: 0.005
    parameterName: lr
    scalingType: LOG
    type: DOUBLE
  - maxValue: 0.001
    minValue: 0.0001
    parameterName: weight_decay
    scalingType: LOG
    type: DOUBLE"""

In [5]:
configs = yaml.load(spec, Loader=yaml.FullLoader)
configs["hpo"]["name"] = "tpe"
configs["hpo"]["max_evals"] = 20
configs['device'] = 'cpu'
configs

{'ensemble': {'name': None},
 'feature': [{'name': 'NormalizeFeatures'}],
 'hpo': {'max_evals': 20, 'name': 'tpe'},
 'models': [{'hp_space': [{'feasiblePoints': '2',
     'parameterName': 'num_layers',
     'type': 'DISCRETE'},
    {'cutFunc': 'lambda x:x[0] - 1',
     'cutPara': ['num_layers'],
     'length': 1,
     'maxValue': [64],
     'minValue': [16],
     'numericalType': 'INTEGER',
     'parameterName': 'hidden',
     'scalingType': 'LOG',
     'type': 'NUMERICAL_LIST'},
    {'maxValue': 0.8,
     'minValue': 0.2,
     'parameterName': 'dropout',
     'scalingType': 'LINEAR',
     'type': 'DOUBLE'},
    {'feasiblePoints': ['leaky_relu', 'relu', 'elu', 'tanh'],
     'parameterName': 'act',
     'type': 'CATEGORICAL'}],
   'name': 'gcn-model'}],
 'trainer': {'hp_space': [{'maxValue': 300,
    'minValue': 100,
    'parameterName': 'max_epoch',
    'scalingType': 'LINEAR',
    'type': 'INTEGER'},
   {'maxValue': 30,
    'minValue': 10,
    'parameterName': 'early_stopping_round',


In [6]:
autoClassifier = AutoNodeClassifier.from_config(configs)

[2022-05-31 09:10:20] WARNING (root/MainThread) Ignore passed dec since enc is a whole model


In [7]:
import logging
# logging.Logger.manager.loggerDict
logging.getLogger().isEnabledFor(logging.WARNING)

True

In [8]:
logging.getLogger().addFilter(lambda s: "Ignore passed dec since enc is a whole model" not in s.getMessage())

In [9]:
# train
if dataset_name in ["cora", "citeseer", "pubmed"]:
    autoClassifier.fit(dataset, time_limit=3600, evaluation_method=[Acc])
else:
    autoClassifier.fit(
        dataset,
        time_limit=3600,
        evaluation_method=[Acc],
        seed=seed,
        train_split=20 * num_classes,
        val_split=30 * num_classes,
        balanced=False,
    )

[2022-05-31 09:10:20] INFO (NodeClassifier/MainThread) Use the default train/val/test ratio in given dataset
HPO Search Phase:



  0%|          | 0/20 [00:00<?, ?it/s]

[2022-05-31 09:10:20] INFO (hyperopt.tpe/MainThread) tpe_transform took 0.073220 seconds
[2022-05-31 09:10:20] INFO (hyperopt.tpe/MainThread) TPE using 0 trials
[2022-05-31 09:10:25] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 275, 'early_stopping_round': 15, 'lr': 0.014164148516981535, 'weight_decay': 0.0006419292885702267}, 'encoder': {'num_layers': 2, 'hidden': [20], 'dropout': 0.7736718727304612, 'act': 'relu'}, 'decoder': {}} acc: 0.808 higher_better


  5%|▌         | 1/20 [00:05<01:42,  5.38s/it]

[2022-05-31 09:10:25] INFO (hyperopt.tpe/MainThread) tpe_transform took 0.002169 seconds
[2022-05-31 09:10:25] INFO (hyperopt.tpe/MainThread) TPE using 1/1 trials with best loss -0.808000
[2022-05-31 09:10:30] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 271, 'early_stopping_round': 27, 'lr': 0.04323531162686548, 'weight_decay': 0.00032750370206197614}, 'encoder': {'num_layers': 2, 'hidden': [34], 'dropout': 0.5387337145728119, 'act': 'tanh'}, 'decoder': {}} acc: 0.81 higher_better


...

[2022-05-31 09:12:13] INFO (hyperopt.tpe/MainThread) tpe_transform took 0.002153 seconds
[2022-05-31 09:12:13] INFO (hyperopt.tpe/MainThread) TPE using 18/18 trials with best loss -0.812000
[2022-05-31 09:12:17] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 293, 'early_stopping_round': 11, 'lr': 0.029907962636370397, 'weight_decay': 0.000526907062533777}, 'encoder': {'num_layers': 2, 'hidden': [34], 'dropout': 0.4846252384428438, 'act': 'relu'}, 'decoder': {}} acc: 0.802 higher_better


 95%|█████████▌| 19/20 [01:57<00:05,  5.10s/it]

[2022-05-31 09:12:18] INFO (hyperopt.tpe/MainThread) tpe_transform took 0.002316 seconds
[2022-05-31 09:12:18] INFO (hyperopt.tpe/MainThread) TPE using 19/19 trials with best loss -0.812000
[2022-05-31 09:12:28] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 197, 'early_stopping_round': 27, 'lr': 0.009982900802243988, 'weight_decay': 0.0007904411595331108}, 'encoder': {'num_layers': 2, 'hidden': [19], 'dropout': 0.38351425365310265, 'act': 'tanh'}, 'decoder': {}} acc: 0.806 higher_better


100%|██████████| 20/20 [02:08<00:00,  6.42s/it]


[2022-05-31 09:12:28] INFO (HPO/MainThread) Best Parameter:
[2022-05-31 09:12:28] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 139, 'early_stopping_round': 21, 'lr': 0.008329372188913789, 'weight_decay': 0.0003416743789200837}, 'encoder': {'num_layers': 2, 'hidden': [47], 'dropout': 0.772354437743668, 'act': 'elu'}, 'decoder': {}} acc: 0.812 higher_better


# KServe

In [4]:
import joblib
import copy
import dill

In [10]:
acf.dataset = None
autoClassifier = acf

In [11]:
with open('autoClassifier.dill', 'wb') as file:
    dill.dump(autoClassifier, file)

In [12]:
with open('autoClassifier.dill', 'rb') as file:
    acf = dill.load(file)

In [26]:
for m in acf.trained_models.values():
    print (type(m.encoder))

<class 'autogl.module.model.pyg.gcn.AutoGCN'>


In [13]:
acf.predict_proba(dataset)

[2022-05-31 13:02:22] INFO (NodeClassifier/MainThread) Ensemble argument on, will try using ensemble model.
[2022-05-31 13:02:22] WARNING (NodeClassifier/MainThread) Cannot use ensemble because no ensebmle module is given. Will use best model instead.


array([[0.17524756, 0.20365748, 0.16183828, ..., 0.09015678, 0.15732759,
        0.06911414],
       [0.0546177 , 0.03481817, 0.77426016, ..., 0.03831783, 0.02858152,
        0.02712895],
       [0.07899787, 0.121613  , 0.44402656, ..., 0.07093763, 0.07331976,
        0.02558529],
       ...,
       [0.14718598, 0.2669439 , 0.014981  , ..., 0.01907321, 0.256131  ,
        0.12125333],
       [0.05522497, 0.05555305, 0.05347392, ..., 0.14460424, 0.04363336,
        0.02152764],
       [0.05620539, 0.0667403 , 0.06415007, ..., 0.12279192, 0.05495729,
        0.02350215]], dtype=float32)

In [84]:
autoClassifier.predict_proba(dataset)

[2022-05-31 12:59:23] INFO (NodeClassifier/MainThread) Ensemble argument on, will try using ensemble model.
[2022-05-31 12:59:23] WARNING (NodeClassifier/MainThread) Cannot use ensemble because no ensebmle module is given. Will use best model instead.


array([[0.17524756, 0.20365748, 0.16183828, ..., 0.09015678, 0.15732759,
        0.06911414],
       [0.0546177 , 0.03481817, 0.77426016, ..., 0.03831783, 0.02858152,
        0.02712895],
       [0.07899787, 0.121613  , 0.44402656, ..., 0.07093763, 0.07331976,
        0.02558529],
       ...,
       [0.14718598, 0.2669439 , 0.014981  , ..., 0.01907321, 0.256131  ,
        0.12125333],
       [0.05522497, 0.05555305, 0.05347392, ..., 0.14460424, 0.04363336,
        0.02152764],
       [0.05620539, 0.0667403 , 0.06415007, ..., 0.12279192, 0.05495729,
        0.02350215]], dtype=float32)

In [30]:
from torch_geometric.datasets import (
    Amazon, Coauthor, Flickr, ModelNet,
    Planetoid, PPI, QM9, Reddit, TUDataset
)
import os

In [32]:
pyg_dataset = Planetoid('./datasets', "Cora")

Processing...
Done!


In [35]:
if hasattr(pyg_dataset, "__data_list__"):
    delattr(pyg_dataset, "__data_list__")
if hasattr(pyg_dataset, "_data_list"):
    delattr(pyg_dataset, "_data_list")

In [36]:
pyg_data = pyg_dataset[0]
pyg_data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [88]:
pyg_data.x.size()

torch.Size([2708, 1433])

In [40]:
type(pyg_data.x), pyg_data.x

(torch.Tensor,
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [41]:
type(pyg_data.y), pyg_data.y

(torch.Tensor, tensor([3, 4, 4,  ..., 3, 3, 3]))

In [104]:
x = getattr(pyg_data, 'train_mask')
type(x), x, x.size()

(torch.Tensor,
 tensor([ True,  True,  True,  ..., False, False, False]),
 torch.Size([2708]))

In [44]:
getattr(pyg_data, 'test_mask')

tensor([False, False, False,  ...,  True,  True,  True])

In [75]:
from autogl.data.graph import GeneralStaticGraphGenerator
from autogl.data import InMemoryStaticGraphSet

In [96]:
        static_graph = GeneralStaticGraphGenerator.create_homogeneous_static_graph(
            {
                'x': pyg_data.x,
                'y': pyg_data.y,
                'train_mask': getattr(pyg_data, 'train_mask'),
                'val_mask': getattr(pyg_data, 'val_mask'),
                'test_mask': getattr(pyg_data, 'test_mask')
            },
            pyg_data.edge_index
        )
    
dataset = InMemoryStaticGraphSet([static_graph])
dataset

In [63]:
!pip install pyTigerGraph

In [64]:
import pyTigerGraph as tg
conn = tg.TigerGraphConnection("http://35.230.92.92", graphname="Cora")

In [117]:
hp = {"batch_size": 64, "num_neighbors": 10, "num_hops": 2, "hidden_dim": 64,
      "num_layers": 2, "dropout": 0.6, "lr": 0.01, "l2_penalty": 5e-4}
infer_loader = conn.gds.neighborLoader(
                                v_in_feats=["x"],
                                v_out_labels=["y"],
                                output_format="PyG",
                                batch_size=hp["batch_size"],
                                num_neighbors=hp["num_neighbors"],
                                num_hops=hp["num_hops"],
                                shuffle=False)

In [118]:
input_nodes = [
      {"primary_id": 7, "type": "Paper"}, 
      {"primary_id": 999, "type": "Paper"}
    ]

data = infer_loader.fetch(input_nodes)
data

Data(edge_index=[2, 16], x=[15, 1433], y=[15], is_seed=[15], primary_id=[15])

In [120]:
data.edge_index

tensor([[ 7, 10, 13, 10, 12, 13,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8],
        [12,  8,  8, 13,  7, 10,  0,  1,  2,  9,  3,  6, 14,  5, 11,  4]])

In [113]:
type(data.x), data.x

(torch.Tensor,
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]))

In [87]:
data.x.size()

torch.Size([15, 1433])

In [69]:
type(data.y), data.y

(torch.Tensor, tensor([2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2]))

In [73]:
#getattr(data, 'train_mask')
x = torch.zeros(10, dtype=torch.bool)
type(x), x

(torch.Tensor,
 tensor([False, False, False, False, False, False, False, False, False, False]))

In [127]:
mask = torch.ones(data.x.size()[0], dtype=torch.bool)

static_graph = GeneralStaticGraphGenerator.create_homogeneous_static_graph(
    {
        'x': data.x.float(),
        'y': data.y,
        'train_mask': mask,
        'val_mask': mask,
        'test_mask': mask
    },
    data.edge_index
)
    
tg_dataset = InMemoryStaticGraphSet([static_graph])
tg_dataset

In [128]:
acf.predict(dataset)

[2022-05-31 17:16:19] INFO (NodeClassifier/MainThread) Ensemble argument on, will try using ensemble model.
[2022-05-31 17:16:19] WARNING (NodeClassifier/MainThread) Cannot use ensemble because no ensebmle module is given. Will use best model instead.


array([1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 5, 5, 5, 4,
       4, 1, 1, 3, 1, 1, 1, 6, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 6, 6, 3, 0, 0,
       0, 0, 5, 0, 0, 0, 3, 0, 0, 6, 0, 6, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5,
       5, 5, 2, 2, 2, 4, 4, 4, 4, 3, 3, 2, 5, 5, 5, 5, 6, 5, 5, 5, 5, 3,
       4, 3, 0, 3, 1, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0,
       3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4,
       6, 6, 5, 6, 6, 0, 5, 5, 5, 0, 5, 0, 4, 0, 3, 3, 3, 2, 3, 1, 3, 3,
       3, 3, 3, 3, 1, 3, 3, 4, 4, 4, 3, 3, 3, 3, 3,

In [129]:
#with torch.no_grad():

acf.predict(tg_dataset, inplaced = False, inplace=True)

[2022-05-31 17:16:20] INFO (NodeClassifier/MainThread) Ensemble argument on, will try using ensemble model.
[2022-05-31 17:16:20] WARNING (NodeClassifier/MainThread) Cannot use ensemble because no ensebmle module is given. Will use best model instead.


array([2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2])

In [62]:
for k, v in autoClassifier.trained_models.items():
    print(k)
    print (type(v))
    print ('---')
    v = copy.deepcopy(v)

decoder: None
early_stopping_round: 21
encoder: <autogl.module.model.pyg.gcn.AutoGCN object at 0x7fd8ac8e3e50>
learning_rate: 0.008329372188913789
max_epoch: 139
optimizer: !!python/name:torch.optim.adam.Adam ''
trainer_name: NodeClassificationFullTrainer
_idx0
<class 'autogl.module.train.node_classification_full.NodeClassificationFullTrainer'>
---


In [64]:
encoder = v.encoder

In [65]:
encoder.hyper_parameters

{'num_layers': 2, 'hidden': [47], 'dropout': 0.772354437743668, 'act': 'elu'}

In [67]:
joblib.dump(encoder.__class__, '__class__.joblib')

['__class__.joblib']

In [71]:
joblib.dump(encoder.model, 'model.joblib')

['model.joblib']

In [73]:
encoder.model.state_dict()

OrderedDict([('_GCN__sequential_encoding_layers.0._convolution.bias',
              tensor([ 0.0604,  0.0159,  0.0130,  0.0507,  0.0272,  0.0204,  0.0408,  0.0055,
                       0.0560,  0.0350,  0.0353,  0.0571,  0.0531,  0.0193,  0.0385,  0.0012,
                       0.0253,  0.0544,  0.0338,  0.0204, -0.0114, -0.0110,  0.0539,  0.0171,
                       0.0172,  0.0174,  0.0236,  0.0258,  0.0165,  0.0072,  0.0200,  0.0148,
                      -0.0128,  0.0157,  0.0704,  0.0419,  0.0165,  0.0085,  0.0197,  0.0097,
                       0.0125, -0.0299,  0.0281,  0.0074, -0.0322,  0.0380,  0.0724])),
             ('_GCN__sequential_encoding_layers.0._convolution.lin.weight',
              tensor([[ 3.3453e-02,  1.0288e-01,  6.3909e-02,  ..., -7.5389e-02,
                        6.8808e-02, -7.8669e-02],
                      [-2.7337e-02,  1.9078e-02, -1.4864e-01,  ..., -4.1355e-02,
                        2.6141e-01, -1.1605e-01],
                      [-1.5863e-03

In [72]:
type(encoder.model)

autogl.module.model.pyg.gcn.GCN

In [42]:
repr(autoClassifier)

'<autogl.solver.classifier.node_classifier.AutoNodeClassifier object at 0x7fd87fe30c10>'

In [13]:
autoClassifier.feature_module

In [14]:

joblib.dump(autoClassifier.feature_module, 'feature_module.joblib')

['feature_module.joblib']

decoder: None
early_stopping_round: 21
encoder: <autogl.module.model.pyg.gcn.AutoGCN object at 0x7fd8ac8e3e50>
learning_rate: 0.008329372188913789
max_epoch: 139
optimizer: !!python/name:torch.optim.adam.Adam ''
trainer_name: NodeClassificationFullTrainer
_idx0
<class 'autogl.module.train.node_classification_full.NodeClassificationFullTrainer'>
---


In [51]:
v.hyper_parameters

{'max_epoch': 139,
 'early_stopping_round': 21,
 'lr': 0.008329372188913789,
 'weight_decay': 0.0003416743789200837}

In [49]:
v.duplicate_from_hyper_parameter(v.)

<bound method NodeClassificationFullTrainer.duplicate_from_hyper_parameter of decoder: None
early_stopping_round: 21
encoder: <autogl.module.model.pyg.gcn.AutoGCN object at 0x7fd8ac8e3e50>
learning_rate: 0.008329372188913789
max_epoch: 139
optimizer: !!python/name:torch.optim.adam.Adam ''
trainer_name: NodeClassificationFullTrainer
>

In [31]:
v.encoder.hyper_parameters

{'num_layers': 2, 'hidden': [47], 'dropout': 0.772354437743668, 'act': 'elu'}

In [37]:
list(autoClassifier.trained_models.keys())

["decoder: None\nearly_stopping_round: 24\nencoder: <autogl.module.model.pyg.gcn.AutoGCN object at 0x7f3b3d4e8df0>\nlearning_rate: 0.0076789463628175415\nmax_epoch: 298\noptimizer: !!python/name:torch.optim.adam.Adam ''\ntrainer_name: NodeClassificationFullTrainer\n_idx0",
 "decoder: None\nearly_stopping_round: 17\nencoder: <autogl.module.model.pyg.gcn.AutoGCN object at 0x7f3c8497c700>\nlearning_rate: 0.01266182115775376\nmax_epoch: 276\noptimizer: !!python/name:torch.optim.adam.Adam ''\ntrainer_name: NodeClassificationFullTrainer\n_idx0"]

In [22]:
bestModel = autoClassifier.get_leaderboard().get_best_model()
bestModel['encoder']

TypeError: string indices must be integers

In [10]:
import joblib